<a href="https://colab.research.google.com/github/rennyatwork/CegepSteFoy_IA/blob/main/08_deep_learning/TransferLearning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Nous allons utiliser le transfer learning pour entrainer un classificateur de fleurs (à partir du jeu de données tf_flowers de TensorFlow) en ré-utilisant le modèle pré-entrainé Xception

1. Importation des packages utiles

In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds

2. Importation des données, ainis que des informations pertinentes sur ces données fournies de base par TensorFlow

In [2]:
dataset, info = tfds.load("tf_flowers",as_supervised=True, with_info=True)

Dl Completed...:   0%|          | 0/5 [00:00<?, ? file/s]

Dataset tf_flowers downloaded and prepared to /root/tensorflow_datasets/tf_flowers/3.0.1. Subsequent calls will reuse this data.


3. Quelques calculs intermédiaires

In [3]:
dataset_size = info.splits["train"].num_examples
print(dataset_size)

class_names = info.features["label"].names
print(class_names)

n_classes = info.features["label"].num_classes
print(n_classes)

3670
['dandelion', 'daisy', 'tulips', 'sunflowers', 'roses']
5


In [4]:
dir(info)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_builder',
 '_builder_or_identity',
 '_features',
 '_fully_initialized',
 '_identity',
 '_info_proto',
 '_metadata',
 '_splits',
 'add_file_data_source_access',
 'add_sql_data_source_access',
 'add_tfds_data_source_access',
 'as_json',
 'as_proto',
 'citation',
 'config_description',
 'config_name',
 'config_tags',
 'data_dir',
 'dataset_size',
 'description',
 'disable_shuffling',
 'download_size',
 'features',
 'file_format',
 'from_proto',
 'full_name',
 'homepage',
 'initialize_from_bucket',
 'initialized',
 'metadata',
 'module_name',
 'name',
 'read_from_directory',
 'redistribution_info',
 'release_notes',
 'set_f

4. Comme le jeu de données tf_flowers ne vient qu'avec un "training data", nous allons ré-importer les données et faire simultanément un split train (70%), test (20%) et validation (10%)

NOTE #1: ici le paramètre "as_supervised=True" sert à indiquer à Keras que l'on souhaite importer les labels (les y) aussi et non-seulement les X

NOTE #2: le paramètre "batch_size=-1" sert à importer les X et les y séparément avec tensorflow dataset

In [5]:
#train_set_raw, test_set_raw, valid_set_raw = tfds.load("tf_flowers",split=["train[:70%]","train[70%:90%]","train[90%:]"], as_supervised=True)

(X_train_raw, y_train_raw), (X_test_raw, y_test_raw), (X_valid_raw, y_valid_raw) = tfds.load("tf_flowers",split=["train[:70%]","train[70%:90%]","train[90%:]"],batch_size=-1, as_supervised=True)

print(X_train_raw.shape, y_train_raw.shape, X_test_raw.shape, y_test_raw.shape, X_valid_raw.shape, y_valid_raw.shape)

(2569, 442, 1024, 3) (2569,) (734, 439, 640, 3) (734,) (367, 441, 640, 3) (367,)


5. Pré-processing #1: adaptater les images de tf_flowers au format requis par le modèle Xception: dans le cas de Xception, les images doivent être de taille 224 x 224

In [6]:
X_train = tf.image.resize(X_train_raw, (224,224))
X_test = tf.image.resize(X_test_raw, (224,224))
X_valid = tf.image.resize(X_valid_raw, (224,224))

print(X_train.shape, X_test.shape, X_valid.shape)

(2569, 224, 224, 3) (734, 224, 224, 3) (367, 224, 224, 3)


6. Pré-processing #2: on transforme les labels en variables catégoriques

In [7]:
from tensorflow.keras.utils import to_categorical

y_train = to_categorical(y_train_raw,num_classes=5)
y_test = to_categorical(y_test_raw,num_classes=5)
y_valid = to_categorical(y_valid_raw,num_classes=5)
print(y_train.shape, y_test.shape, y_valid.shape)

(2569, 5) (734, 5) (367, 5)


In [8]:
y_train

array([[0., 0., 1., 0., 0.],
       [0., 0., 0., 1., 0.],
       [0., 0., 0., 1., 0.],
       ...,
       [1., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0.],
       [1., 0., 0., 0., 0.]], dtype=float32)

7. Pré-processing #3: importer Xception ainsi que son application de pré-processing des inputs (notamment, avec Xception, les valeurs de chaque pixels doivvent être entre -1 et +1, mais tout sera traité par la fonction de pré-processing)

In [9]:
from tensorflow.keras.applications.xception import Xception
from tensorflow.keras.applications.xception import preprocess_input

X_train = preprocess_input(X_train)
X_test = preprocess_input(X_test)
X_valid = preprocess_input(X_valid)

print(X_train.shape, X_test.shape, X_valid.shape)

(2569, 224, 224, 3) (734, 224, 224, 3) (367, 224, 224, 3)


8. En transfer learning, il est souvent d'usage de faire du "data augmentation", surtout lorsque le nombre d'exemple est petit

In [10]:
data_augmentation = tf.keras.Sequential([
    tf.keras.layers.RandomFlip(mode="horizontal",seed=42),
    tf.keras.layers.RandomRotation(factor=0.05,seed=42),
    tf.keras.layers.RandomContrast(factor=0.2,seed=42)
])

9. Définition du modèle via l'API Functionnal de Keras, le tout en utilisant le modèle Xception (sans le top, c'est-à-dire sans la dernière couche, car nous allons remplacer cette dernière couche par nos propres couches (avgPooling et SoftMax en output))

In [11]:
base_model = Xception(weights="imagenet", include_top=False)

avg = tf.keras.layers.GlobalAveragePooling2D()(base_model.output)

output = tf.keras.layers.Dense(n_classes,activation="softmax")(avg)

model = tf.keras.Model(inputs = base_model.input, outputs=output)

83683744/83683744 [==============================] - 1s 0us/step


10. Pour commencer, nous n'allons pas ré-entrainer les poids et biais de Xception (nous allons commencer par uniquement entrainer les poids de notre Softmax final)

In [12]:
for layer in base_model.layers:
  layer.trainable=False

11. Compiler le modèle

In [13]:
optimizer = tf.keras.optimizers.SGD(learning_rate=0.1, momentum=0.9)

model.compile(loss="categorical_crossentropy",optimizer=optimizer,metrics=["accuracy"])

12. Entrainer le modèle pour quelques epochs (NOTE: vous pouvez rouler le modèle sur un GPU, ex (Colab): Runtime/Change runtime type/T4 GPU pour avoir plus de rapidité, mais attention, car le modèle est gourmand sur les unités de calcul de Google)

In [ ]:
history = model.fit(X_train, y_train, batch_size=32 , validation_data=(X_valid,y_valid),epochs=3)

Epoch 1/3
81/81 [==============================] - 612s 8s/step - loss: 1.3655 - accuracy: 0.6022 - val_loss: 1.0271 - val_accuracy: 0.7466
Epoch 2/3
27/81 [=========>....................] - ETA: 6:10 - loss: 1.0336 - accuracy: 0.7025

13. Évaluation de l'ajustement du modèle (avec les métriques classiques: loss, accuracy)

In [1]:
loss, acc = model.evaluate(X_test, y_test, verbose=0)
print('Accuracy: %.3f' % acc)

NameError: ignored

14. Si on atteint un plateau d'accuracy en entrainant seulement la dernière couche, c'est souvent signe qu'on peut commencer à entrainer aussi quelques couches du réseau pré-entrainer en les débloquant, par exemple, ici on débloque toutes les couches à partir de la 56e

In [ ]:
for layer in base_model.layers[56:]:
  layer.trainable=True

optimizer = tf.keras.optimizers.SGD(learning_rate=0.01, momentum=0.9)

model.compile(loss="categorical_crossentropy",optimizer=optimizer,metrics=["accuracy"])

history = model.fit(X_train,y_train,batch_size=32 , validation_data=(X_valid,y_valid),epochs=5)

Epoch 1/3
81/81 [==============================] - 1447s 18s/step - loss: 0.6197 - accuracy: 0.7789 - val_loss: 0.3144 - val_accuracy: 0.9074
Epoch 2/3
81/81 [==============================] - 1431s 18s/step - loss: 0.1976 - accuracy: 0.9280 - val_loss: 0.3496 - val_accuracy: 0.8937
Epoch 3/3
81/81 [==============================] - 1438s 18s/step - loss: 0.0867 - accuracy: 0.9673 - val_loss: 0.3773 - val_accuracy: 0.8910


13. Ré-évaluation de la performance du modèle

In [ ]:
loss, acc = model.evaluate(X_test, y_test, verbose=0)
print('Accuracy: %.3f' % acc)

NameError: ignored